In [ ]:
# coding: utf-8
# this will download audioset youtube samples based on the audioset segments
# 
# balanced
# unbalanced
# eval

import numpy as np
import pandas as pd
import re
import csv
import datetime
import os
import wave


seg_path = '../labels/audioset/segments/'
#segment = 'balanced_train'
#segment = 'unbalanced_train'
segment = 'eval'

## Lees csv-file in
labels = pd.read_csv(seg_path+segment+"_segments.csv", skiprows=3, 
                          quotechar='"', skipinitialspace = True, header=None, 
                          names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

In [ ]:
labels.head()

In [ ]:
## op pcloud is er een symlink naar de data folder
# downloaded segments go to

target_file_path = "../../../data/audio/serval-data/raw/youtube-downloads/"+segment+"/"
#target_file_path = "./"+segment+"/"

# welke custom serval wordt er getraned
custom_target = "jungle" #"urban"

# we need a list of custom class labels
mid_to_label = pd.read_csv("../labels/custom/"+custom_target+"/class_labels_indices_jungle_v2.csv", sep=";")

mid_to_label

In [ ]:
## Lijst van alle labels (nieuwe) en alle combi's
all_labels = []
label_combinations = []

In [ ]:
# define functions
# find unique set of labels in custom target

def getLabels(mid_str):
    ## Maak lijst van m-id's
    mid_list = mid_str.split(',')
    labels = []
    
    ## Voor elk m-id, vind labels, (if any labels: add to label list)
    for mid in mid_list:
        if (mid_to_label.loc[mid_to_label["mid"] == mid, "mid"].any()):
            labels.append(mid_to_label.loc[mid_to_label["mid"] == mid, "index"].values[0])
    
    ## Return unique set of labels
    return set(labels)

In [ ]:
# Download wav-files van youtube

def downloadWav(vid_to_mid, path):
    # now we actually download the segments as they are no conversion yet
    mids = vid_to_mid["positive_labels"]
    
    labels = getLabels(mids)
    #print('aantal labels : ',len(labels))
    
    
    if(len(labels)>0):
        # Voeg labels toe aan labellijst en combilijst
        all_labels.extend(labels)
        label_combinations.append(labels)
        
        url = vid_to_mid["YTID"]
        start_sec = vid_to_mid["start_seconds"]
        start_time = str(datetime.timedelta(seconds=start_sec)) + '.00'
        file_path = path + "vid" + url 
  
        if not os.path.exists(file_path):
            # we only download the 10 second segment as is
            cmd = 'youtube-dl -f "bestaudio" -o "' + file_path + '.%(ext)s" --extract-audio  --postprocessor-args "-ss ' + start_time + ' -t 00:00:10.00" "https://www.youtube.com/v/' + url + '"'
            #print("CMD="+cmd)
            os.system(cmd)
        else:
             print('File already exists!')

In [ ]:
# add progressbar
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing


num_cores = multiprocessing.cpu_count()

# run in parralel
results = Parallel(n_jobs=num_cores)(delayed(downloadWav)(labels.loc[i], target_file_path) for i in tqdm(range(labels.shape[0])))

# not paralel
#for i in tqdm(range(labels.shape[0])):
# for i in range(labels.shape[0]):
#     downloadWav(labels.loc[i], target_file_path)

LET OP:
Voordat onderstaande wordt gerund, moet balanced, unbalanced en eval allemaal gedownload zijn.
Dan pas kan er worden geteld hoeveel voorkomens er zijn.
Idee om misschien van tevoren al die drie samen te voegen tot 1 csv?

In [ ]:
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [ ]:
## Print per label aantal voorkomens
from collections import Counter

cnt_labels = Counter(all_labels)

for i in cnt_labels.most_common(100):
    print(mid_to_label.loc[mid_to_label['index']==i[0]]['display_name'].values[0], ' : ', str(i[1]))

In [ ]:
## Houd individuele voorkomens bij (dus aantal keer dat label voorkomt zonder andere labels)
ind_occur = [0 for x in range(len(mid_to_label.index2.unique()))] 

## Geef meest voorkomende combinaties
cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    
    ## Maak van string weer een lijst (zie vorige cel)
    label_list = label_combination[0].split("_")
    
    ## Code die alleen individuele labels telt
    if(len(label_list)==1):
        ind_occur[int(label_list[0])] = label_combination[1]

In [ ]:
## Code die kolommen toevoegt aan label-csv en deze in de forloop vult

## Aantal gewenste samples per label: 2500
target = 2500

## Aantal voorkomens
mid_to_label["total"] = 0
## Aantal voorkomens alleen
mid_to_label["ind_total"] = 0
## Proportie waarmee gedown/upsampled moet worden (rekening houdend met combi van labels)
mid_to_label["target_ind"] = 0

for i in range(len(ind_occur)):
    mid_to_label.loc[mid_to_label['index2']==i,"total"] = cnt_labels[i]
    mid_to_label.loc[mid_to_label['index2']==i,"ind_total"] = ind_occur[i]
    if(ind_occur[i] > 0):
        mid_to_label.loc[mid_to_label['index2']==i,"target_ind"] = (ind_occur[i]-(cnt_labels[i]-target))/ind_occur[i]

In [ ]:
mid_to_label